# Agent 1: Data Reader

Attach data file to code interpreter action . Pass code interpreter tool to Agent service agent. Agents will use code interpreter to run python code to analyse historical consumption data.

In [1]:
# load environment variables from the .env file and import the necessary modules
from dotenv import load_dotenv
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.ai.projects.models import FilePurpose
from azure.identity import DefaultAzureCredential
from pathlib import Path

load_dotenv()

True

## Create Agent

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Create a project client

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

In [4]:
# Upload a file to the project
file = project_client.agents.upload_file_and_poll(
    file_path="./data/consumption.csv", purpose=FilePurpose.AGENTS
)
print(f"Uploaded file, file ID: {file.id}")

Uploaded file, file ID: assistant-xrBo5E8fmCiLRWECdYCphpaw


In [5]:
# Create a code interpreter tool and add the file to it
code_interpreter = CodeInterpreterTool(file_ids=[file.id])

In [6]:
instruction = """
# Role
You are an AI agent specialized using python code to extract insights from the historical electricity consumption data.
"""

agent = project_client.agents.create_agent(
    model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    name="data_analysis",
    instructions=instruction,
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
    # Parameters
    temperature=0.5,
    top_p=0.95,
    
)

print(f"Created agent, agent ID: {agent.id}")

Created agent, agent ID: asst_C6cSSXmi8efpyMy6xHJv1x2P


In [22]:
import time
from azure.ai.projects.models import RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

# Create thread
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="Hello,what is my average consumption so far"
)
print(f"Created message, ID: {message.id}")

# Create and process assistant run in thread with tools
 # create and execute a run
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

# # delete the original file from the agent to free up space (note: this does not delete your version of the file)
# project_client.agents.delete_file(file.id)
# print("Deleted file")

# print the messages from the agent
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages.data[0].content[0]['text']['value']}")




Created thread, ID: thread_wVBkwLO15hobLLM5RsjwtSWa
Created message, ID: msg_5LLoLxSAe2SA9uQQm5GVSvJL
Run finished with status: RunStatus.COMPLETED
Messages: Your average electricity consumption so far is approximately 470.58 units.

If you have any other questions or need further analysis, please let me know!


In [32]:
# save the created image files
for image_content in messages.image_contents:
  print(f"Image File ID: {image_content.image_file.file_id}")
  file_name = f"{image_content.image_file.file_id}_image_file.png"
  project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
  print(f"Saved image file to: {Path.cwd() / file_name}")

Image File ID: assistant-wxGzNblv90OQMroqQWk9sVrb
Saved image file to: c:\Users\shilpajain\OneDrive - Microsoft\codes\gbbai-workshops-main\multi-agent\single_agent\assistant-wxGzNblv90OQMroqQWk9sVrb_image_file.png
